<a href="https://colab.research.google.com/github/AkshitAggarwal/TSAI_EVA5B2_Phase1/blob/main/Session_04/Assignment_Session04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils 
import torch.autograd
import torch.optim as optim
import torchvision.transforms as transforms

#Train Dataset and Train Dataloader

In [2]:
train_data = torchvision.datasets.EMNIST(
    root ='\.bin',
    download = True,
    train = True,
    split = 'byclass',
    transform = transforms.ToTensor()
)

train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 32,
    shuffle = True,
)

Extracting \.bin/EMNIST/raw/emnist.zip to \.bin/EMNIST/raw
Processing byclass


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Processing bymerge
Processing balanced
Processing letters
Processing digits
Processing mnist
Done!


In [3]:
print(len(train_data), len(train_loader)) #Total no. of Training images and Training batches(Each batch has 32 images)

697932 21811


In [4]:
print(train_data.train_labels.bincount()) #Individual counts of all classes. Total classes: 62

tensor([34585, 38374, 34203, 35143, 33535, 31416, 34232, 35754, 33946, 33847,
         6407,  3878, 10094,  4562,  4934,  9182,  2517,  3152, 11946,  3762,
         2468,  5076,  9002,  8237, 24983,  8347,  2605,  5073, 20764,  9820,
        12602,  4637,  4695,  2771,  4743,  2701, 10033,  5159,  2854, 10177,
        24631,  2561,  3687,  8738,  2725,  1896,  2491, 15318,  2645, 11418,
         2749,  2448,  2994, 14105,  2699, 18262,  2830,  2910,  2697,  2822,
         2365,  2725])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:48: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [5]:
type(train_data)

torchvision.datasets.mnist.EMNIST

In [6]:
batch_iterator = next(iter(train_loader))
type(batch_iterator)

list

In [7]:
images, labels = batch_iterator
print(images.shape, labels.shape)

torch.Size([32, 1, 28, 28]) torch.Size([32])


In [8]:
sample = next(iter(train_data))
print(type(sample))
image, label = sample
print(image.shape, label)

<class 'tuple'>
torch.Size([1, 28, 28]) 35


#Network Architecture

In [20]:
class neural_net(nn.Module):
  def __init__(self):
    super(neural_net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3) #insize= 1x28x28; outsize= 10x26x26; RF= 3
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3) #insize= 10x26x26; outsize= 10x24x24; RF= 5
    self.maxp1 = nn.MaxPool2d(kernel_size = 2, stride = 2) #insize= 10x24x24; outsize= 10x12x12; RF= 10
    self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3) #insize= 10x12x12; outsize= 20x10x10; RF= 12
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3) #insize= 20x10x10; outsize= 20x8x8; RF= 14
    self.maxp2 = nn.MaxPool2d(kernel_size = 2, stride = 2) #insize= 20x8x8; outsize= 20x4x4; RF= 28
    self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3) #insize= 20x4x4; outsize= 30x2x2; RF= 30
    self.outconv = nn.Conv2d(in_channels=30, out_channels=62, kernel_size=1) #insize = 30x2x2; outsize= 62x2x2; RF= 30
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) #insize = 62x2x2; outsize= 62x1x1; RF=?
  
  def forward(self, t):
    t = self.maxp1(F.relu(self.conv2(F.relu(self.conv1(t)))))
    t = self.maxp2(F.relu(self.conv4(F.relu(self.conv3(t)))))
    t = self.outconv(F.relu(self.conv5(t)))
    t = self.avgpool(t)
    # t = t.view(-1, 62)
    return F.softmax(t, dim = 1)

In [21]:
network = neural_net()
print(network)

neural_net(
  (conv1): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
  (maxp1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(20, 20, kernel_size=(3, 3), stride=(1, 1))
  (maxp2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(20, 30, kernel_size=(3, 3), stride=(1, 1))
  (outconv): Conv2d(30, 62, kernel_size=(1, 1), stride=(1, 1))
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
)


In [22]:
torch.set_grad_enabled(False)

In [23]:
pred = network(image.unsqueeze(dim=0)) #Image is a rank-3 tensor, but our model expects a rank-4 'batch-like' tensor.
# print(pred.squeeze().shape) 

In [24]:
print(pred.shape)
# pred.eq(label)
print(pred.squeeze())

torch.Size([1, 62, 1, 1])
tensor([0.0192, 0.0179, 0.0180, 0.0160, 0.0176, 0.0145, 0.0134, 0.0173, 0.0174,
        0.0136, 0.0165, 0.0162, 0.0185, 0.0174, 0.0186, 0.0152, 0.0134, 0.0164,
        0.0191, 0.0162, 0.0165, 0.0186, 0.0166, 0.0135, 0.0195, 0.0142, 0.0142,
        0.0164, 0.0139, 0.0172, 0.0168, 0.0149, 0.0170, 0.0182, 0.0170, 0.0148,
        0.0179, 0.0141, 0.0176, 0.0145, 0.0133, 0.0141, 0.0152, 0.0135, 0.0165,
        0.0180, 0.0143, 0.0175, 0.0182, 0.0148, 0.0172, 0.0157, 0.0145, 0.0191,
        0.0164, 0.0139, 0.0150, 0.0174, 0.0141, 0.0141, 0.0166, 0.0151])


In [25]:
print(torch.sum(pred.squeeze()))

tensor(1.)


In [32]:
pred.argmax(dim=1).squeeze() == label

tensor(False)

Testing with a bunch of images. 

In [33]:
batch = next(iter(train_loader))
print(type(batch), len(batch))
images, labels = batch
print(images.shape, labels.shape)

<class 'list'> 2
torch.Size([32, 1, 28, 28]) torch.Size([32])


In [34]:
preds = network(images)
print(preds.shape)

torch.Size([32, 62, 1, 1])


In [36]:
# for param in network.parameters():
  # print(param.shape)
for name, param in network.named_parameters():
  print(name, '\t\t', param.shape)

conv1.weight 		 torch.Size([10, 1, 3, 3])
conv1.bias 		 torch.Size([10])
conv2.weight 		 torch.Size([10, 10, 3, 3])
conv2.bias 		 torch.Size([10])
conv3.weight 		 torch.Size([20, 10, 3, 3])
conv3.bias 		 torch.Size([20])
conv4.weight 		 torch.Size([20, 20, 3, 3])
conv4.bias 		 torch.Size([20])
conv5.weight 		 torch.Size([30, 20, 3, 3])
conv5.bias 		 torch.Size([30])
outconv.weight 		 torch.Size([62, 30, 1, 1])
outconv.bias 		 torch.Size([62])


#Shifting the model onto a GPU

In [40]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = neural_net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
            Conv2d-2           [-1, 10, 24, 24]             910
         MaxPool2d-3           [-1, 10, 12, 12]               0
            Conv2d-4           [-1, 20, 10, 10]           1,820
            Conv2d-5             [-1, 20, 8, 8]           3,620
         MaxPool2d-6             [-1, 20, 4, 4]               0
            Conv2d-7             [-1, 30, 2, 2]           5,430
            Conv2d-8             [-1, 62, 2, 2]           1,922
 AdaptiveAvgPool2d-9             [-1, 62, 1, 1]               0
Total params: 13,802
Trainable params: 13,802
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.14
Params size (MB): 0.05
Estimated Total Size (MB): 0.19
---------------------------------------------

In [41]:
#Changing Autograd Universally back to True
torch.set_grad_enabled(True)